# Definizione delle funzini

In [11]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import lxml.html
from lxml import etree
import xml.etree.ElementTree as ET
import string
import numpy as np
import time
import gzip
import re

sito_url = 'www.bestbuy.com'

def AbsolutePaths(tags):
    tmp=''
    for tag in tags:
        tmp+=tag.tag+'/'
    return str(tmp)

def load_gz(file):
    #contiene come chiave il nome del file e come valore la pagina html
    with gzip.open('dataset/'+ sito_url +'/pages/'+ file +'.gz', 'rb') as f:
        file_content = f.read() #html
    return file_content
  
def xpath_generator_gz(C): #C è una lista di liste [[nome.html, url], ... , ...]
  X = []
  for lista in C:
    link = lista[1]
    #carico la pagina html
    html = load_gz(lista[0])
    #analizza l'elemento html della pagina
    tree = lxml.html.fromstring(html)
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, weight del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

def xpath_generator(C): #C è una lista
  X = []
  for link in C:
    #ritorna la get
    link = ''.join(link)
    link = link[:len(link)-1]    
    response = requests.get(link)
    #ritorna l'elemento html della pagina
    tree = lxml.html.fromstring(response.text)
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, weight del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

#Ricerca la frequency e la frequency del contenuto di un certo xpath
#e la cardinalità dell'insieme di tutti i contenuti associati a quell'xpath
def find_f(url, xpath, content, lista):
  frequency = 1
  f_content = 1
  #t_i è l'insieme di tutti i contenuti associati a xpath
  t_i = [content]
  #lista dei url già visti nella ricerca
  url_visti = []
  url_visti.append(url)
  for new_path in lista:
    #se allo stesso xpath ho un contenuto diverso da quelli che già ho lo inserisco in t_i
    if new_path[1] == xpath and new_path[2] not in t_i:
      t_i.append(new_path[2])
    #calcolo della frequency e di f_content
    if new_path[0] not in url_visti:
      if new_path[1] == xpath:
        url_visti.append(new_path[0])
        frequency += 1
        if new_path[2] == content:
          f_content += 1
  return frequency, f_content, len(t_i)

#lista = [(url,xpath,contenuto,weight), (....), (....), .... ]
def weight_xpath(lista, num_pagine):
  for xpath in lista:
    #prendo xpath
    x = xpath[1]
    url = xpath[0]
    contenuto = xpath[2]
    #calcolo la frequency, la frequency del contenuto e la cardinalità di t_i
    frequency, f_content, t_i = find_f(url, x, contenuto, lista)
    #calcolo della informatività
    info = 1 - (f_content / (num_pagine * t_i))
    #calcolo del weight del xpath
    weight = frequency * info
    #inserisco il weight nella lista
    xpath[3] = weight
    
def sum_weights_page(X):
  url_weights = {}
  for e in X:
    k = e[0]
    if k not in url_weights.keys():
      url_weights[k] = e[3]
    else:
      url_weights[k] += e[3]
  return url_weights

#Trova la pagina (ritornando il suo url) che ha massima la somma dei weights di tutti i suoi xpath
def find_p(X):
  #creo un dizionario che ha come chiave l'url di una pagina e come valore la somma di tutti i weights dei suoi xpath
  weighted_pages = sum_weights_page(X)
  weight_max = 0
  page_max = ''
  #ricerco la pagina che ha valore massimo nel dizionario creato precedentemente
  for k in weighted_pages.keys():
    weight_page = weighted_pages[k]
#     print(k, weight_page)
    if weight_page > weight_max:
      page_max = k
      weight_max = weight_page
#   print('\n')
  #ritorno l'url della pagina trovata
  return page_max, weight_max

#Crea un dizionario con chiave xpath e valore il contenuto
def build_dict(X, p):
  d = {}
  for e in X:
    if e[0] == p:
      if e[1] not in d:
        xpath = e[1]
        content = e[2]
        d[xpath] = content
  return d

#Rimuove la pagina da X e gli xpath associati ad essa
def remove(X, p):
  #nuova lista da riportare
  X1 = []
  dict_xpath_content = build_dict(X,p)
  for e in X:
    #se l'xpath è contenuto in una pagina diversa da p
    #e ha un xpath non presente in p allora lo aggiungo a X1
    if e[0] != p and e[1] not in dict_xpath_content:
      X1.append(e)
  return X1

# ============= MAIN =============
def page_annotation(C, K):
  #X = xpath_generator(C)
  start = time.time()
  X = xpath_generator_gz(C)
  print('XPath totali: ' + str(len(X)))
  weight_xpath(X, len(C))
  S = []
  weights = []
  while X != [] and len(S) <= K:      
    p, weight = find_p(X)
    S.append(p)
    print(str(len(S))+'° ' + p + ' - weight: ' + str(weight) +'\n')
    weights.append(weight)
    X = remove(X, p)
  stop = time.time()
  print(str((stop - start)/60) + ' min') #time
  return S, X, weights

# Carico il dataset
Il dataset preso in considerazione è quello di 'bestbuy'

In [2]:
nome_url = []
with open('dataset/'+ sito_url +'/_id2name.txt', 'r') as file:
    for line in file:
        id_file = re.search('[0-9]+.html',line).group()
        nome_url.append([id_file])
i=0
with open('dataset/'+ sito_url +'/_id2url.txt', 'r') as file:
    for line in file:
        id_url = re.split('\t', line)[1]
        id_url = id_url.replace('\n','')
        lista = nome_url[i]
        lista.append(id_url)
        nome_url[i] = lista
        i+=1

In [12]:
primi100 = nome_url[:100]

# Start

In [13]:
S, x, pesi = page_annotation(primi100,5)

print('XPath rimanenti:',len(x)) #stampiamo il numero di xpath rimanenti

XPath totali: 58503
1° http://www.bestbuy.com/site/lenovo-yoga-2-2-in-1-11-6-touch-screen-laptop-intel-pentium-4gb-memory-500gb-hard-drive-silver/3297045.p;jsessionid=3E6D02131286BB42249A70868018A3AC.bbolsp-app04-179?id=1219090034631&skuId=3297045&st=categoryid$abcat0502000&cp=1&lp=5 - Peso: 52656.73994846078

2° http://www.bestbuy.com/site/asus-refurbished-11-6-vivobook-ultrabook-4-gb-memory-320-gb-hard-drive/1310162150.p;jsessionid=5589BBCFA01AE0FE136D3C99BD9A6CDC.bbolsp-app04-108?id=mp1310162150&skuId=1310162150&st=categoryid$abcat0502000&cp=22&lp=8#tab=buyingOptions - Peso: 691.6043571428571

3° http://www.bestbuy.com/site/hp-15-6-elitebook-notebook-4-gb-memory-500-gb-hard-drive/1308851424.p;jsessionid=563D812A6BD53499E8AE0A33B4E10F88.bbolsp-app04-164?id=mp1308851424&skuId=1308851424&st=categoryid$pcmcat247400050000&cp=31&lp=13#tab=buyingOptions - Peso: 112.95960000000001

4° http://www.bestbuy.com/site/dell-14-latitude-notebook-4-gb-memory/1311178471.p;jsessionid=14068F3B03018F7BF

# Prove per le metriche dell'algoritmo
precizion e recall